In [67]:
import pandas as pd
import re
from pattern.text.en import singularize
from spellchecker import SpellChecker
spell = SpellChecker()
from tqdm import tqdm_notebook as tqdm
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.remove('other')
import string
import enchant
import pickle
d = enchant.Dict("en_US")
tqdm().pandas()

## Finding main objects

In [7]:
def datafram_from_csv(file_path):
    df = pd.read_csv(file_path,encoding='latin1')
    return df

In [8]:
def get_concat_data(list_of_files):
    list_of_df = [datafram_from_csv(i) for i in list_of_files]
    df = pd.concat(list_of_df, axis=0, ignore_index=True)
    return df

In [44]:
df = get_concat_data(['data/Insta_labeled_data/image_labels_0plus_to_10_.csv','data/Insta_labeled_data/image_labels_10plus_to_40.csv','data/Insta_labeled_data/image_labels_40plus.csv'])

In [45]:
df.head()

,id_image,label_image
0,https://scontent.cdninstagram.com/hphotos-xaf1...,Person Person\nCelebrity\nClothes\nText Pers...
1,https://scontent.cdninstagram.com/hphotos-xap1...,Person Celebrity celebritÃ Person
2,https://scontent.cdninstagram.com/hphotos-xap1...,Nature\nPerson\nClothes\nFood Person\nFood P...
3,https://scontent.cdninstagram.com/hphotos-xaf1...,Cartoon Other art dontknow
4,https://scontent.cdninstagram.com/hphotos-xap1...,Clothes Clothes Clothes


In [57]:
duplicate_map = {
 'acres':'adress',
 'advertisment':'advertising',
    'ad':'advertising',
    'animals':'animal',
    'bicycle':'bike',
    'broke':'broken',
    'buildingslandmark':'building',
    'celebrating':'celebration',
    'cigar':'cigarette',
    'cloth':'clothe',
    'clubbing':'club',
    'musical':'music',
    'paintdrawing':'painting',
    'art':'painting',
    'artwork':'painting',
    'tv':'television',
    'showing':'show',
    'dontknow': '',
    'figurine':'figure',
    'graffiti':'graphics',
    'glas':'glass',
    'golden':'gold',
    'gras':'grass',
    'home':'house',
    'instruments':'instrument',
    'kids':'kid',
    'loading':'load',
    'motorbike':'motorcycle',
    'sex':'sensual',
    'sexy':'sensual'
}

In [49]:
objects = [spell.correction(singularize(j.lower())) for i in tqdm(df['label_image'].tolist()) for j in re.split(' |\n', i) if j!='']

In [70]:
objects = [word for word in objects if word not in stop_words]
objects = [s.translate(str.maketrans('','',string.punctuation)) for s in objects]
objects = [duplicate_map[i] if i in duplicate_map.keys() else i for i in objects]
objects = [word for word in objects if word not in ['','ð¸ð½ñ\x82ðµñ\x80ñ\x8cðµñ\x80','ñ\x80ðµðºð»ð°ð¼ð°','load','picture','photo','nice','content','love']]

In [71]:
unique_objects = set(objects)
len(unique_objects)

392

In [72]:
unique_objects

{'accessory',
 'action',
 'address',
 'adress',
 'advertising',
 'advice',
 'airplane',
 'airport',
 'alcohol',
 'animal',
 'apple',
 'area',
 'areawiev',
 'arena',
 'arm',
 'army',
 'article',
 'artist',
 'atelier',
 'auditorium',
 'automotive',
 'baby',
 'backpack',
 'bag',
 'bamboo',
 'banner',
 'barro',
 'baseball',
 'basketball',
 'bathroom',
 'beauty',
 'bed',
 'bedroom',
 'behavior',
 'best',
 'bike',
 'bikini',
 'black',
 'body',
 'book',
 'boot',
 'booty',
 'boy',
 'broken',
 'bu',
 'building',
 'burned',
 'businesscard',
 'buy',
 'cake',
 'camera',
 'campaign',
 'car',
 'card',
 'care',
 'cartoon',
 'cat',
 'cd',
 'celebration',
 'celebrity',
 'cellphone',
 'chair',
 'chandelier',
 'christmas',
 'church',
 'cigarette',
 'city',
 'clock',
 'clothe',
 'cloud',
 'club',
 'coffee',
 'colm',
 'company',
 'computer',
 'concert',
 'concussion',
 'cool',
 'coral',
 'couple',
 'craft',
 'crashed',
 'crayon',
 'cup',
 'currency',
 'dance',
 'demonstration',
 'desktop',
 'destruction',


In [74]:
final_objects =[]
extras = []
result = sorted([(x, objects.count(x)) for x in unique_objects], key=lambda y: y[1])
for elem in result:
    if elem[1]>=5:
        final_objects.append(elem[0])
    else:
        extras.append(elem[0])

In [76]:
len(final_objects)

42

In [78]:
with open('data/DATA_I_MADE/final_objects.pkl', 'wb') as fp:
    pickle.dump(final_objects, fp)

with open('data/DATA_I_MADE/extra_onjects.pkl', 'wb') as fp:
    pickle.dump(extras, fp)

## Objects in each row

In [89]:
with open('data/DATA_I_MADE/final_objects.pkl', 'rb') as fp:
    final_objects = pickle.load(fp)

with open('data/DATA_I_MADE/extra_onjects.pkl', 'rb') as fp:
    extras = pickle.load(fp)

In [84]:
def pre_process_class(objects_str):
    objects = [spell.correction(singularize(j.lower())) for j in re.split(' |\n', objects_str) if j!='']
    objects = [word for word in objects if word not in stop_words]
    objects = [s.translate(str.maketrans('','',string.punctuation)) for s in objects]
    objects = [duplicate_map[i] if i in duplicate_map.keys() else i for i in objects]
    objects = [word for word in objects if word not in ['','ð¸ð½ñ\x82ðµñ\x80ñ\x8cðµñ\x80','ñ\x80ðµðºð»ð°ð¼ð°','load','picture','photo','nice','content','love']]
    objects = [w for w in objects if w in final_objects]
    return list(set(objects))

In [96]:
def get_object_in_row(row):
    return pre_process_class(row['label_image'])

In [101]:
df['objects'] = df.progress_apply(lambda row: get_object_in_row(row),axis=1)

In [103]:
df.head()

,id_image,label_image,objects
0,https://scontent.cdninstagram.com/hphotos-xaf1...,Person Person\nCelebrity\nClothes\nText Pers...,"[text, clothe, celebrity, person]"
1,https://scontent.cdninstagram.com/hphotos-xap1...,Person Celebrity celebritÃ Person,"[celebrity, person]"
2,https://scontent.cdninstagram.com/hphotos-xap1...,Nature\nPerson\nClothes\nFood Person\nFood P...,"[nature, clothe, food, person]"
3,https://scontent.cdninstagram.com/hphotos-xaf1...,Cartoon Other art dontknow,"[cartoon, painting, other]"
4,https://scontent.cdninstagram.com/hphotos-xap1...,Clothes Clothes Clothes,[clothe]


In [104]:
with open('data/DATA_I_MADE/objects_df.pkl', 'wb') as fp:
    pickle.dump(df, fp)

## Finding mapping

In [112]:
def closure_clean_url_link(row,link_column_name):
    def clean_url_link(row):
        return row[link_column_name].replace('\n','').replace('\r','')
    return clean_url_link(row)

Objects file links

In [113]:
with open('data/DATA_I_MADE/objects_df.pkl', 'rb') as fp:
    df = pickle.load(fp)

In [114]:
df.head()

,id_image,label_image,objects
0,https://scontent.cdninstagram.com/hphotos-xaf1...,Person Person\nCelebrity\nClothes\nText Pers...,"[text, clothe, celebrity, person]"
1,https://scontent.cdninstagram.com/hphotos-xap1...,Person Celebrity celebritÃ Person,"[celebrity, person]"
2,https://scontent.cdninstagram.com/hphotos-xap1...,Nature\nPerson\nClothes\nFood Person\nFood P...,"[nature, clothe, food, person]"
3,https://scontent.cdninstagram.com/hphotos-xaf1...,Cartoon Other art dontknow,"[cartoon, painting, other]"
4,https://scontent.cdninstagram.com/hphotos-xap1...,Clothes Clothes Clothes,[clothe]


In [115]:
df['id_image'] = df.progress_apply(lambda row: closure_clean_url_link(row,'id_image'),axis=1)

In [116]:
img_urls_object_file = df['id_image'].tolist()

final dataset file links

In [117]:
with open('data/DATA_I_MADE/final_csv_with_image_names.pkl', 'rb') as fp:
    df_labeled = pickle.load(fp)

In [118]:
df_labeled['img_url'] = df_labeled.progress_apply(lambda row: closure_clean_url_link(row,'img_url'),axis=1)

In [119]:
img_url_labeled_file = df_labeled['img_url'].tolist()

images folder link mapping

In [120]:
list_of_files = ['data/instagram_images/namesall_info_media_ids_to_collect_0plus_to_10.txt','data/instagram_images/namesall_info_media_ids_to_collect_10plus_to_40.txt','data/instagram_images/namesall_info_media_ids_to_collect_40plus.txt']
list_of_df = [pd.read_csv(i, sep=' ',names=['image_id','links']) for i in list_of_files]
df_image_folder = pd.concat(list_of_df, axis=0, ignore_index=True)

In [121]:
df_image_folder['links'] = df_image_folder.progress_apply(lambda row: closure_clean_url_link(row,'links'),axis=1)

In [122]:
img_urls_image_file = df_image_folder['links'].tolist()

Finding matches

In [124]:
len(set(img_urls_object_file)),len(set(img_url_labeled_file)),len(set(img_urls_image_file))

(2242, 2216, 1921)

In [125]:
len(set(img_urls_object_file)&set(img_url_labeled_file))

2213

In [126]:
len(set(img_urls_object_file)&set(img_urls_image_file))

138

Adding label to object rows

In [128]:
df

,id_image,label_image,objects
0,https://scontent.cdninstagram.com/hphotos-xaf1...,Person Person\nCelebrity\nClothes\nText Pers...,"[text, clothe, celebrity, person]"
1,https://scontent.cdninstagram.com/hphotos-xap1...,Person Celebrity celebritÃ Person,"[celebrity, person]"
2,https://scontent.cdninstagram.com/hphotos-xap1...,Nature\nPerson\nClothes\nFood Person\nFood P...,"[nature, clothe, food, person]"
3,https://scontent.cdninstagram.com/hphotos-xaf1...,Cartoon Other art dontknow,"[cartoon, painting, other]"
4,https://scontent.cdninstagram.com/hphotos-xap1...,Clothes Clothes Clothes,[clothe]
5,https://scontent.cdninstagram.com/hphotos-xfa1...,Person Person\nClothes\nShoes Person\nClothes,"[shoe, clothe, person]"
6,https://scontent.cdninstagram.com/hphotos-xaf1...,Text Text Text,[text]
7,https://scontent.cdninstagram.com/hphotos-xap1...,Person Person\nClothes Person,"[clothe, person]"
8,https://scontent.cdninstagram.com/hphotos-xpf1...,Person Person Nature\nPerson,"[nature, person]"
9,https://scontent.cdninstagram.com/hphotos-xpa1...,Person\nOther music Celebrity Person,"[music, celebrity, other, person]"
